In [1]:
#imports
using Distributions;
using Random;
using DataFrames;
using CSV;
using Statistics;
using LinearAlgebra;
using Flux;
using Flux: params, 
            Dense, 
            Chain, 
            glorot_normal, 
            normalise, 
            Optimiser,
            train!;


function nn_estimation(file_name)
        
    #parameters of the function
    d = Int(3); #number of dimensions
    N = 100; #number of timesteps of SDE evolution
    T = Int(1);  #ttm
    h = T / N #size of a timestep
    
    #drift and volatility terms
    alpha_1, alpha_2, alpha_3 = 10., 14., 8/3
    beta = ones(3)*0.15
    
    batch_samples = Int(1024); #total samples in batch
    mc_test_batch = Int(500); #number of samples in testing batches
    
    function X_sampler(batch_samples)
        x_1 = rand(Uniform(0.5,2.5),(1,batch_samples))
        x_2 = rand(Uniform(8,10),(1,batch_samples))
        x_3 = rand(Uniform(10,12),(1,batch_samples))
        x = Array(vcat(x_1,x_2,x_3))
    end
    
    function mu(x)
        x_1 = Array(transpose(x[1,:]))
        x_2 = Array(transpose(x[2,:]))
        x_3 = Array(transpose(x[3,:]))
        mu_1 = alpha_1 .* (x_2 .- x_1)
        mu_2 = alpha_2 .* x_1 .- x_2 .- x_3 
        mu_3 = x_1 .*x_2 .- alpha_3 .*x_3
        mu_x = Array(vcat(mu_1,mu_2,mu_3))
        return mu_x
    end

    mc_samples = Int(20000); #number of samples to take for Monte-Carlo approximation
    mc_exp_rounds = Int(5); #number of times to repeat MC for the error average
    
    learning_rate = Float16(0.001); #initial learning rate
    learn_rate_decrease = 250000; #how frequently to decay learning rate

    train_steps = Int(750000); #total number of training epochs
    err_step = Int(25000); #after how many training steps to compare errors
    
 #   #standard normalisation function
 #   function norm_ab(y)
 #       mid_point = (a + b) / 2
 #       y_norm(y) = (y .- mid_point) ./ (b-a)
 #       return mapslices(y_norm,y;dims =1)
 #   end
        
    function x_sde(x,batch_samples)
        for n in 1:N
            norm_mu = sqrt.(sum(mu(x).^2;dims =1))
            ind = ifelse.(norm_mu .> (N/T),0,1)
            #sampled brownian motion
            eps = rand(Normal(0,1),(3,batch_samples))
            x = x .+ (mu(x).*h).*ind .+ beta .* eps
        end
        return x
    end


    function x_phi(x::Array) #function to use with FK expectation
        sum(x .^2; dims =1)
    end;

    #initialise error file and create row headers
    df_row = DataFrame(step = "step",
                        l1_errs="l1_errs",l2_errs="l2_errs",li_errs="li_errs",
                        rel_l1_errs="rel_l1_errs",rel_l2_errs="rel_l2_errs",rel_li_errs="rel_li_errs",
                        t_nn="t_nn",t_mc ="t_mc")
    
    CSV.write(file_name, df_row, append = true);
    
    #calculate errors and write to file
    function k_iter_output(t_nn, k)
        
        #generate mc data
        function mc_sampler(x_val, mc_test_batch)
            x_mc_store = zeros((1,mc_test_batch))
            for _ in 1:mc_samples
                x_mc_store += Array((x_phi(x_sde(x_val,mc_test_batch))))
            end
            phi_mc = x_mc_store ./ mc_samples;
            return Array(phi_mc)
        end
           
        #find the expected error vs mc samples
        #initial errors for finding the mean
        t_mc = 0 
        l1_errs,l2_errs,li_errs = 0.,0.,0.
        rel_l1_errs, rel_l2_errs, rel_li_errs = 0., 0., 0.
            
        for _ in 1:mc_exp_rounds
            
            x_val = X_sampler(mc_test_batch)

            #run through testmode NN for comparison
            X_0 = Array(normalise(x_val))
            
            u_i = m(X_0)    

            #take mc samples
            t_start = time()
            mc_i = mc_sampler(x_val,mc_test_batch)
            t_end = time()
            t_mc += t_end - t_start
            u_ref = abs.(max.(mc_i,1e-8))
            
            #calculate and output errors
            errs = vec(abs.(u_i - mc_i))
            l1_errs += mean(errs)
            l2_errs += mean(errs.^2)
            li_errs = max(li_errs, maximum(errs))
            rel_errs = errs ./ u_ref
            rel_l1_errs += mean(rel_errs)
            rel_l2_errs += mean(rel_errs.^2)    
            rel_li_errs = max(rel_li_errs,maximum(rel_errs))
            
        end

        #find means
        t_mc = t_mc / mc_exp_rounds
        l1_errs,l2_errs = l1_errs/mc_exp_rounds, sqrt(l2_errs/mc_exp_rounds)
        rel_l1_errs,rel_l2_errs = rel_l1_errs/mc_exp_rounds, sqrt(rel_l2_errs/mc_exp_rounds)  
        
        #write to file
        df_row = DataFrame(step = k,
                            l1_errs=l1_errs,l2_errs=l2_errs,li_errs=li_errs,
                            rel_l1_errs=rel_l1_errs,rel_l2_errs=rel_l2_errs,rel_li_errs=rel_li_errs,
                            t_nn=t_nn,t_mc =t_mc)
        
        CSV.write(file_name, df_row, append = true)
    end
    
    
    function generate_training_data(X_init,x_sde,x_phi)    
        X_0 = Array(normalise(X_init))
        X_sde = x_sde(X_init,batch_samples)
        y_train = x_phi(X_sde)
        return [(X_0,y_train)]
    end

    #define network layers
    input = Dense(d, d + 20, identity; 
                           bias = false, 
                           init = glorot_normal)

    hidden = Dense(d + 20, d + 20, identity;
                            bias = false,
                            init = glorot_normal)

    #no activation on the last layer
    output = Dense(d + 20,1,identity)

    batch_norm_layer = BatchNorm(d + 20, tanh;
                                            initβ = zeros, 
                                            initγ = ones,
                                            ϵ = 1e-6, 
                                            momentum = 0.01)
    
    #define network architecture
    m = Chain(input,
           #     batch_norm_layer,
                hidden,
          #      batch_norm_layer,
        #        hidden,
        #        batch_norm_layer,
                output)
    
    #loss function = 
    loss(u,v) = mean((m(u) - v).^2)
    
    ps = Flux.params(m)

    opt = Optimiser(ExpDecay(learning_rate,0.1,learn_rate_decrease,1e-8),ADAM()) #optimiser

    #set to train mode
    trainmode!(m)
    
    #generate initial training data
    X_init = X_sampler(batch_samples)
    data = generate_training_data(X_init,x_sde,x_phi)
    
    k_iter_output(0, 0) #compare with MC at this stage

    #start training time counter
    t_nn_start = time()
    
    for k in 1:train_steps
        
        #generate new training data
        X_init = X_sampler(batch_samples)
        data = generate_training_data(X_init,x_sde,x_phi)

        #learning step
        train!(loss,ps,data,opt)

        #output the errors and timings at these steps
        if mod(k,err_step) == 0 
            
            t_nn_end = time()
            t_nn = t_nn_end - t_nn_start #timer for the training steps
            testmode!(m)
            k_iter_output(t_nn, k) #compare with MC at this stage
            t_nn_start = time() #start new training timer
            trainmode!(m) #set back train mode
            
        end
    end
    
    print("Output ready")

end


nn_estimation("stochastic_lorenz_results_2.csv")

Output ready